In [3]:
!pip install mediapipe --user

In [4]:
import mediapipe as mp

In [5]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import time

In [6]:
mp_holistic =  mp.solutions.holistic
mp_drawing =  mp.solutions.drawing_utils

In [7]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results 


In [8]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
    

In [9]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color =(80,110,10), thickness = 1, circle_radius = 1),
                             mp_drawing.DrawingSpec(color = (80,256,121), thickness = 1, circle_radius =1)) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color =(80,110,10), thickness = 1, circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (80,256,121), thickness = 1, circle_radius =2)) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color =(80,110,10), thickness = 1, circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (80,256,121), thickness = 1, circle_radius =2)) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color =(80,110,10), thickness = 1, circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (80,256,121), thickness = 1, circle_radius =2)) # Draw right hand connections
    

In [ ]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image, results = mediapipe_detection(frame,holistic)
         
        #Draw landmarks
        draw_styled_landmarks(image, results)

        cv2.imshow('OpenCV Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()


## KeyPoint Extraction 

In [10]:
def extract_keypoints(results):
    pose =  np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose ,face ,lh, rh])

## Setup Folders for Collection 

In [11]:
path = "/Users/nick8/Desktop/hsiaoping"
DATA_PATH = os.path.join(path,'MP_Data')
actions = np.array(['hello','thanks','peace'])
no_sequences = 30
sequence_length = 30

In [12]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH ,action, str(sequence)))
        except:
            pass
        

## Collect Keypoint value Training and Testing


In [11]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    
    for action in actions:
        
        for sequence in range(no_sequences):
            
            for frame_num in range(sequence_length):
                
                ret, frame = cap.read()

                image, results = mediapipe_detection(frame,holistic)

                #Draw landmarks
                draw_styled_landmarks(image, results)
                
                #Apply collection logic 
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
               
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(DATA_PATH, action , str(sequence), str(frame_num))
                    np.save(npy_path, keypoints)
                    cv2.imshow('OpenCV Feed', image)
        
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
    cap.release()
    cv2.destroyAllWindows()

## Preprocess Data and Create Labels and Features 

In [13]:
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical


In [14]:
label_map = {label:num for num,label in enumerate(actions)}
label_map

{'hello': 0, 'thanks': 1, 'peace': 2}

In [24]:
sequences =[] 
labels = []

for action in actions:
    for sequence in range(no_sequences):
        window = []
        
        for frame_num in range(1,sequence_length-1):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(window)
        sequences.append(window)
        labels.append(label_map[action])
print("sequences")
print(sequences[0][0][0])
print("labels")
print(labels)

sequences
[[...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...]]
labels
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [ ]:
X = np.array(sequences) # input data

In [ ]:
y = to_categorical(labels).astype(int)  # labels

In [21]:
X_train,X_test,y_train, y_test =  train_test_split(X,y,test_size = 0.05)

NameError: name 'X' is not defined

## Build and Train LSTM Neural Network

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense 
from tensorflow.keras.callbacks import TensorBoard

In [17]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir= log_dir)

In [18]:
model = Sequential()
model.add(LSTM(64,return_sequences = True, activation = 'relu',input_shape = (30,1662)))
model.add(LSTM(128, return_sequences = True, activation = 'relu' ))
model.add(LSTM(64,return_sequences = False , activation = 'relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation ='relu'))
model.add(Dense(actions.shape[0],activation = 'softmax'))

In [23]:
model.compile(optimizer = 'Adam', loss ='categorical_crossentropy', metrics = ['categorical_accuracy'])

In [20]:
model.fit(X_train,y_train, epochs = 200, callbacks = [tb_callback])

NameError: name 'X_train' is not defined

## Make predictions

In [ ]:
res = model.predict(X_test)

## Save Weights

In [ ]:
model.save('action.h5')
model.load_weights('action.h5')

## Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score 

In [ ]:
yhat = model.predict(X_test)

ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(y_hat, axis = 1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

## Test in real time

In [ ]:
colors = [(245,117,16), (117,245,16),(16,117,245)]

def prob_viz(res,actions, input_frame , colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100) , 90+num*40), colors[num],-1 )
        cv2.putText(output_frame, actions[num] ,(0,85+num*40) , cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    
    return output_frame
    

In [ ]:
sequence = []
sentence = []
threshold = 0.4
predictions = []

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image, results = mediapipe_detection(frame,holistic)
         
        #Draw landmarks
        draw_styled_landmarks(image, results)
        
        #2.Prediction logic
        keypoints =  extract_keypoints(results)
        sequence.append(keypoint)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence,axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
        
        if np.unique(predictions[-10:])[0] == np.argmax(res):
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
        
        if len(sentence) > 5:
            sentence = sentence[-5:]
        
        image = prob_viz(res , actions , image , colors)
        
        cv2.rectangle(image , (0,0) ,(640,40) ,(245,117,16),-1)
        cv2.putText(image, ' '.join(sentence),(3,30), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            

        cv2.imshow('OpenCV Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()